# Lab02: Frequent itemset mining

- Student ID: 21127705
- Student name: Từ Phước Toàn

**How to do your homework**


You will work directly on this notebook; the word `TODO` indicate the parts you need to do.

You can discuss ideas with classmates as well as finding information from the internet, book, etc...; but *this homework must be your*.

**How to submit your homework**

Before submitting, rerun the notebook (`Kernel` ->` Restart & Run All`).

Then create a folder named `ID` (for example, if your ID is 1234567, then name the folder `1234567`) Copy file notebook to this folder, compress and submit it on moodle.

**Contents:**

- Frequent itemset mining.

# 1. Preliminaries
## This is how it all started ...
- Rakesh Agrawal, Tomasz Imielinski, Arun N. Swami: Mining Association Rules between Sets of Items in Large Databases. SIGMOD Conference 1993: 207-216
- Rakesh Agrawal, Ramakrishnan Srikant: Fast Algorithms for Mining Association Rules in Large Databases. VLDB 1994: 487-499

**These two papers are credited with the birth of Data Mining**
## Frequent itemset mining (FIM)

Find combinations of items (itemsets) that occur frequently.
## Applications
- Items = products, transactions = sets of products someone bought in one trip to the store.
$\Rightarrow$ items people frequently buy together.
    + Example: if people usually buy bread and coffee together, we run a sale of bread to attract people attention and raise price of coffee.
- Items = webpages, transactions = words. Unusual words appearing together in a large number of documents, e.g., “Brad” and “Angelina,” may indicate an interesting relationship.
- Transactions = Sentences, Items = Documents containing those sentences. Items that appear together too often could represent plagiarism.

## Transactional Database
A transactional database $D$ consists of $N$ transactions: $D=\left\{T_1,T_2,...,T_N\right\}$. A transaction $T_n \in D (1 \le n \le N)$ contains one or more items and that $I= \left\{ i_1,i_2,…,i_M \right\}$ is the set of distinct items in $D$, $T_n \subset I$. Commonly, a transactional database is represented by a flat file instead of a database system: items are non-negative integers, each row represents a transaction, items in a transaction separated by space.

Example: 

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 

30 31 32 

33 34 35 

36 37 38 39 40 41 42 43 44 45 46 

38 39 47 48 

38 39 48 49 50 51 52 53 54 55 56 57 58 

32 41 59 60 61 62 

3 39 48 

63 64 65 66 67 68 



# Definition

- Itemset: A collection of one or more items.
    + Example: {1 4 5}
- **k-itemset**: An itemset that contains k items.
- Support: Frequency of occurrence of an itemset.
    + Example: From the example above, item 3 appear in 2 transactions so its support is 2.
- Frequent itemset: An itemset whose support is greater than or equal to a `minsup` threshold

# The Apriori Principle
- If an itemset is frequent, then all of its subsets must also be frequent.
- If an itemset is not frequent, then all of its supersets cannot be frequent.
- The support of an itemset never exceeds the support of its subsets.
$$ \forall{X,Y}: (X \subseteq Y) \Rightarrow s(X)\ge s(Y)$$


# 2. Implementation


## The Apriori algorithm
Suppose:

$C_k$ candidate itemsets of size k.

$L_k$ frequent itemsets of size k.

The level-wise approach of Apriori algorithm can be descibed as follow:
1. k=1, $C_k$ = all items.
2. While $C_k$ not empty:
    3. Scan the database to find which itemsets in $C_k$ are frequent and put them into $L_k$.
    4. Use $L_k$ to generate a collection of candidate itemsets $C_{k+1}$ of size k+1.
    5. k=k+1.

### Import library

In [25]:
from collections import defaultdict

### Read data
First we have to read data from database

In [26]:

def readData(path):
    """
    Parameters
    --------------------------
        path: path of database D.
         
    --------------------------
    Returns
        data: a dictionary for representing database D
                 - keys: transaction tids
                 - values: itemsets.
        s: support of distict items in D.
    """
    data={}
    s=defaultdict(lambda: 0) # Initialize a dictionary for storing support of items in I.  
    with open(path,'rt') as f:
        tid=1;
        for line in f:
            itemset=set(map(int,line.split())) # a python set is a native way for storing an itemset.
            for item in itemset:  
                s[item]+=1     #Why don't we compute support of items while reading data?
            data[tid]= itemset
            tid+=1
    
    return data, s

### Tree Projection

**I gave you pseudo code of Apriori algorithm above but we implement Tree Projection. Tell me the differences of two algorithms.**


**TODO:**

In [27]:
def joinset(a, b):
    '''
    Parameters
    -------------------
        2 itemsets a and b (of course they are at same branch in search space)

    -------------------
    return
        ret: itemset generated by joining a and b
    '''
    # TODO (hint: this function will be called in generateSearchSpace method.):
    ret = a[:-1] + [a[-1], b[-1]]
    return ret

class TP:
    def __init__(self, data=None, s=None, minSup=None):
        self.data = data
        self.s = {}

        for key, support in sorted(s.items(), key=lambda item: item[1]):
            self.s[key] = support
        # TODO: why should we do this, answer it at the markdown below?

        self.minSup = minSup
        self.L = {}  # Store frequent itemsets mined from database
        self.runAlgorithm()

    def initialize(self):
        """
        Initialize search space at first step
        --------------------------------------
        We represent our search space in a tree structure
        """
        tree = {}

        search_space = {}
        for item, support in self.s.items():
            search_space[item] = {}

            search_space[item]['itemset'] = [item]
            ''' 
            python set does not remain elements order
            so we use a list to extend it easily when create new itemset 
            but why we store itemset in data by a python set???? '''
            # TODO: study about python set and its advantages,
            # answer at the markdown below.

            search_space[item]['pruned'] = False
            # TODO:
            # After finish implementing the algorithm tell me why should you use this
            # instead of delete item directly from search_space and tree.

            search_space[item]['support'] = support

            tree[item] = {}
            '''
            Why should i store an additional tree (here it called tree)? 
            Answer: This really help in next steps.

            Remember that there is always a big gap from theory to practicality
            and implementing this algorithm in python is not as simple as you think.
            '''

        return tree, search_space

    def computeItemsetSupport(self, itemset):

        '''Return support of itemset'''
        # TODO (hint: this is why i use python set in data)
        support = 0
        for T in self.data.values():
            if (itemset.issubset(T) == True):
                support = support + 1
        return support

    def get_sub_tree(self, k, tree, search_space, itter_node):
        if k == 0:
            return search_space[itter_node]['support']
        subtree = search_space[itter_node]
        for node in subtree.keys():
            k-=1
            self.get_sub_tree(k,tree,search_space,node)


    def prune(self, k, tree, search_space):

        '''
        In this method we will find out which itemset in current search space is frequent
        itemset then add it to L[k]. In addition, we prune those are not frequent itemsets.
        '''
        if self.L.get(k) is None: self.L[k] = []
        # TODO
        itemsList = list(tree.keys())
        for i in itemsList:
            items = search_space[i]['itemset']
            itemset = set(items)
            supp = self.computeItemsetSupport(set(itemset))
            if supp >= self.minSup:
                self.L[k].append((items, supp))
                search_space[i]['support'] = supp
            else:
                search_space[i]['pruned'] = True


    def generateSearchSpace(self, k, tree, search_space):
        '''
        Generate search space for exploring k+1 itemset. (Recursive function)
        '''
        items = list(tree.keys())
        ''' print search_space.keys() you will understand  
         why we need an additional tree, '''
        l = len(items)
        self.prune(k, tree, search_space)
        if l == 0: return  # Stop condition
        for i in range(l - 1):
            sub_search_space = {}
            sub_tree = {}
            a = items[i]
            if search_space[a]['pruned']: continue

            for j in range(i + 1, l):
                b = items[j]
                search_space[a][b] = {}
                tree[a][b] = {}
                # You really need to understand what am i doing here before doing work below.
                # (Hint: draw tree and search space to draft).

                # TODO:
                # First create newset using join set
                newset = joinset(search_space[a]['itemset'], search_space[b]['itemset'])
                
                # Second add newset to search_space
                search_space[a][b]['itemset'] = newset
                search_space[a][b]['pruned'] = False
                
            #  Generate search_space for k+1-itemset
            sub_search_space = search_space[a]
            sub_tree = tree[a]
            self.generateSearchSpace(k + 1, sub_tree, sub_search_space)

    def runAlgorithm(self):
        tree, search_space = self.initialize()  # generate search space for 1-itemset
        self.generateSearchSpace(1, tree, search_space)

    def miningResults(self):
        return self.L

Ok, let's test on a typical dataset `chess`.

In [28]:
data, s= readData('chess.txt')

In [29]:
#
a=TP(data=data,s=s, minSup=3000)
print(a.miningResults())

{1: [([48], 3013), ([56], 3021), ([66], 3021), ([34], 3040), ([62], 3060), ([7], 3076), ([36], 3099), ([60], 3149), ([40], 3170), ([29], 3181), ([52], 3185), ([58], 3195)], 2: [([48, 52], 3002), ([48, 58], 3012), ([56, 29], 3006), ([56, 52], 3016), ([56, 58], 3020), ([66, 60], 3021), ([66, 29], 3013), ([66, 52], 3010), ([66, 58], 3020), ([34, 40], 3017), ([34, 29], 3036), ([34, 52], 3031), ([34, 58], 3039), ([62, 60], 3014), ([62, 40], 3045), ([62, 29], 3045), ([62, 52], 3049), ([62, 58], 3060), ([7, 60], 3031), ([7, 40], 3050), ([7, 29], 3069), ([7, 52], 3065), ([7, 58], 3075), ([36, 60], 3052), ([36, 40], 3073), ([36, 29], 3084), ([36, 52], 3088), ([36, 58], 3098), ([60, 40], 3124), ([60, 29], 3136), ([60, 52], 3138), ([60, 58], 3148), ([40, 29], 3155), ([40, 52], 3159), ([40, 58], 3169), ([29, 52], 3170), ([29, 58], 3180), ([52, 58], 3184)], 3: [([48, 52, 58], 3001), ([56, 29, 52], 3001), ([56, 29, 58], 3005), ([56, 52, 58], 3015), ([66, 60, 29], 3013), ([66, 60, 52], 3010), ([66, 6

### Answer questions here:
**Why don't we compute support of items while reading data?**

If we calculate the support of items when reading the data, we won't know which items support needs to be calculated.Candidate sets will be generated when Apriori algorithm's running so we only need to calculate the support for those sets.

**why should we do sort**

We sort support in ascending order of 's' so that when creating search_space and tree, the nodes of the tree will be generated in that order. Nodes with lower support, which lead to pruning, will be positioned at the front, allowing the algorithm to skip over all pruned nodes to start generating candidate sets from the non-pruned nodes located later.

**study about python set and its advantages ?**

A Python set is an unordered collection of unique elements. It is defined by enclosing the elements in curly braces {}. The primary advantage of using a Python set includes:

Uniqueness: A set ensures that all elements inside it are unique. If you try to add a duplicate element to a set, it won't raise an error; instead, the duplicate element will simply be ignored.

Fast Membership Testing: Checking for membership (i.e., whether an element is present in the set) is very efficient with sets. The average time complexity for membership testing in a set is O(1), making it much faster than lists (O(n)) for this operation.

Mathematical Operations: Sets support various mathematical set operations like union, intersection, difference, and symmetric difference, which can be very useful in many programming scenarios.

Immutability of Elements: While the set itself is mutable (i.e., you can add or remove elements), the elements inside a set must be immutable (e.g., numbers, strings, tuples). This ensures that the elements can be hashed, which is necessary for efficient set operations.

Efficient Removal and Addition: Adding and removing elements from a set is very efficient, with an average time complexity of O(1).

**After finish implementing the algorithm tell me why should you use this? Instead of delete item directly from search_space and tree.**

We don't directly delete from search_space and tree, but rather mark them as pruned within the search_space, for two reasons:

- We can know exactly where the search branch is cut off, avoiding loss of this information.
- Deleting an element from a dictionary is a time-consuming operation.

**Apriori algorithm and Tree Projection, tell me the differences of two algorithms.**

Apriori Algorithm:

- Based on the Apriori principle.

- Generates candidate itemsets.

- Requires multiple scans of the database.

- Uses pruning strategies to reduce the search space.

- Generally consumes more memory.

Tree Projection:

- Utilizes the FP-tree structure.

- Constructs the FP-tree with a single scan.

- Grows frequent patterns by projecting the FP-tree.

- Counts support directly from the FP-tree.

- Uses path merging and conditional pattern base pruning.

- Typically consumes less memory.



# 3. Churn analysis

In this section, you will use frequent itemset mining technique to analyze `churn` dataset (for any purposes). 

*Remember this dataset is not represented as a transactional database, first thing that you have to do is transforming it into a flat file.  

In [30]:
# TODO
# YOUR CODE HERE
import pandas as pd
# Use pandas to read the data file
df = pd.read_csv('churn.txt')
df

,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,...,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False.
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False.
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False.
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False.
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3328,AZ,192,415,414-4276,no,yes,36,156.2,77,26.55,...,126,18.32,279.1,83,12.56,9.9,6,2.67,2,False.
3329,WV,68,415,370-3271,no,no,0,231.1,57,39.29,...,55,13.04,191.3,123,8.61,9.6,4,2.59,3,False.
3330,RI,28,510,328-8230,no,no,0,180.8,109,30.74,...,58,24.55,191.9,91,8.64,14.1,6,3.81,2,False.
3331,CT,184,510,364-6381,yes,no,0,213.8,105,36.35,...,84,13.57,139.2,137,6.26,5.0,10,1.35,2,False.


Numeric features Pre-processing:

In [31]:
correlationMatrix = df.drop(['Phone','State', "Int'l Plan", 'VMail Plan', 'Churn?'], axis=1).corr()
'''
Drop all the features that are not a numeric type. Build a correlation matrix to find out the correlation between thoose numeric features. With that way we 
can reduce some unecessery feature for our future analysis.
'''
correlationMatrix

,Account Length,Area Code,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls
Account Length,1.000000,-0.012463,-0.004628,0.006216,0.038470,0.006214,-0.006757,0.019260,-0.006745,-0.008955,-0.013176,-0.008960,0.009514,0.020661,0.009546,-0.003796
Area Code,-0.012463,1.000000,-0.001994,-0.008264,-0.009646,-0.008264,0.003580,-0.011886,0.003607,-0.005825,0.016522,-0.005845,-0.018288,-0.024179,-0.018395,0.027572
VMail Message,-0.004628,-0.001994,1.000000,0.000778,-0.009548,0.000776,0.017562,-0.005864,0.017578,0.007681,0.007123,0.007663,0.002856,0.013957,0.002884,-0.013263
Day Mins,0.006216,-0.008264,0.000778,1.000000,0.006750,1.000000,0.007043,0.015769,0.007029,0.004323,0.022972,0.004300,-0.010155,0.008033,-0.010092,-0.013423
Day Calls,0.038470,-0.009646,-0.009548,0.006750,1.000000,0.006753,-0.021451,0.006462,-0.021449,0.022938,-0.019557,0.022927,0.021565,0.004574,0.021666,-0.018942
Day Charge,0.006214,-0.008264,0.000776,1.000000,0.006753,1.000000,0.007050,0.015769,0.007036,0.004324,0.022972,0.004301,-0.010157,0.008032,-0.010094,-0.013427
Eve Mins,-0.006757,0.003580,0.017562,0.007043,-0.021451,0.007050,1.000000,-0.011430,1.000000,-0.012584,0.007586,-0.012593,-0.011035,0.002541,-0.011067,-0.012985
Eve Calls,0.019260,-0.011886,-0.005864,0.015769,0.006462,0.015769,-0.011430,1.000000,-0.011423,-0.002093,0.007710,-0.002056,0.008703,0.017434,0.008674,0.002423
Eve Charge,-0.006745,0.003607,0.017578,0.007029,-0.021449,0.007036,1.000000,-0.011423,1.000000,-0.012592,0.007596,-0.012601,-0.011043,0.002541,-0.011074,-0.012987
Night Mins,-0.008955,-0.005825,0.007681,0.004323,0.022938,0.004324,-0.012584,-0.002093,-0.012592,1.000000,0.011204,0.999999,-0.015207,-0.012353,-0.015180,-0.009288


We can detect some abnormalities from this matrix. 
1.	The correlation between ‘Day Mins’ feature and ‘Day Charge’ feature is 1.00. Which means Day Mins is a derived features from Day Charge or Day Charge is a derived features from Day Mins.  We can conclude: Day Charge -> Day Mins or Day Mins -> Day Charge. Which means we can drop one of these features out and keep the other feature. I decided to drop Day Charge feature.
2.	The correlation between ‘Eve Mins’ feature and ‘Eve Charge’ feature is 1.00. Which means Eve Mins is a derived features from Eve Charge or Eve Charge is a derived features from Eve Mins.  We can conclude: Eve Charge -> Eve Mins or Eve Mins -> Eve Charge. Which means we can drop one of these features out and keep the other feature. I decided to drop Eve Charge feature.
3.	The correlation between ‘Night Mins’ feature and ‘Night Charge’ feature is 1.00. Which means Night Mins is a derived features from Night Charge or Night Charge is a derived features from Night Mins.  We can conclude: Night Charge -> Night Mins or Night Mins -> Night Charge. Which means we can drop one of these features out and keep the other feature. I decided to drop Night Charge feature.


In [32]:
df = df.drop(['Day Charge', 'Eve Charge', 'Night Charge', 'Intl Charge'], axis=1)

Categorical Features Pre-processing:

In [33]:
'''
According to Churn.txt data we can identify that ‘Phone’ feature is a unique feature with every record of the data. 
Therefore, we can use ‘Phone’ feature as an identified feature for every record (primary key) 
so we have to drop this feature out.
'''
df = df.drop(['Phone'], axis=1)

In [34]:
def transformToHorizontalDB(df, fName):
    with open(fName, 'w') as f:
        for record in df.to_dict("records"):
            for feature, value in record.items():
                feature = feature.replace(" ", "")
                item = "{0}={1} ".format(feature,value)
                f.write(item)
            f.write('\n')
'''
This function transform the database into a transactional database
'''
churn = df.loc[df["Churn?"] == "True."]
transformToHorizontalDB(churn, "churnHorizontal.txt")

In [35]:
def readDataChurn(path):
    data = {}
    s = defaultdict(lambda: 0)
    with open(path, 'rt') as f:
        tid = 1
        for line in f:
            itemset = set(map(str, line.split()))
            for item in itemset:  
                s[item] += 1
            data[tid] = itemset
            tid += 1
    return data, s
'''
Modify readData function into readDataChurn function that can read string type data
'''
churnData, churnS = readDataChurn('churnHorizontal.txt')

In [36]:
minSp = 35*len(churnData)//100
'''
Accroding to this paper https://www.jatit.org/volumes/Vol66No3/22Vol66No3.pdf page 835 said that minSp should be higher than 30% and the number of Trancsactions 
should be below 10000 in order for Apriori has a better performace. So i deciced to choose minSupport = 35%

'''
FIM = TP(data=churnData, s=churnS, minSup=minSp)
churnRes = FIM.miningResults()
'''
We only display the result which has Churn?=True
'''
for key, value in churnRes.items():
    for itemset, support in value:
        if "Churn?=True." in itemset:
            res = "Itemset: {0}, support: {1}".format(itemset,support)
            print(res)

Itemset: ['Churn?=True.'], support: 483
Itemset: ['AreaCode=415', 'Churn?=True.'], support: 236
Itemset: ["Int'lPlan=no", 'Churn?=True.'], support: 346
Itemset: ['VMailPlan=no', 'Churn?=True.'], support: 403
Itemset: ['VMailMessage=0', 'Churn?=True.'], support: 403
Itemset: ['AreaCode=415', "Int'lPlan=no", 'Churn?=True.'], support: 174
Itemset: ['AreaCode=415', 'VMailPlan=no', 'Churn?=True.'], support: 202
Itemset: ['AreaCode=415', 'VMailMessage=0', 'Churn?=True.'], support: 202
Itemset: ["Int'lPlan=no", 'VMailPlan=no', 'Churn?=True.'], support: 302
Itemset: ["Int'lPlan=no", 'VMailMessage=0', 'Churn?=True.'], support: 302
Itemset: ['VMailPlan=no', 'VMailMessage=0', 'Churn?=True.'], support: 403
Itemset: ['AreaCode=415', 'VMailPlan=no', 'VMailMessage=0', 'Churn?=True.'], support: 202
Itemset: ["Int'lPlan=no", 'VMailPlan=no', 'VMailMessage=0', 'Churn?=True.'], support: 302
